http://nikgrozev.com/2015/07/01/reshaping-in-pandas-pivot-pivot-table-stack-and-unstack-explained-with-pictures/

In [1]:
import numpy as np
import pandas as pd

In [7]:
df = pd.DataFrame()
df['Item'] = ['Item0', 'Item0', 'Item1', 'Item1']
df['CType'] = ['Gold', 'Bronze', 'Gold', 'Silver']
df['USD'] = ['1$', '2$', '3$', '4$']
df['EU'] = ['1€', '2€', '3€', '4€']

In [8]:
df

,Item,CType,USD,EU
0,Item0,Gold,1$,1€
1,Item0,Bronze,2$,2€
2,Item1,Gold,3$,3€
3,Item1,Silver,4$,4€


In [9]:
df.pivot(index = 'Item', columns = 'CType', values = 'USD')

CType,Bronze,Gold,Silver
Item,,,
Item0,2$,1$,NaN
Item1,NaN,3$,4$


Pandas creates multi-index when 'values' isn't specified

In [11]:
df.pivot(index = 'Item', columns = 'CType')

USD                 EU            
CType Bronze Gold Silver Bronze Gold Silver
Item                                       
Item0     2$   1$    NaN     2€   1€    NaN
Item1    NaN   3$     4$    NaN   3€     4€

# pivot_table vs. pivot

- If a table has duplicate values for the same index-column combination, **pivot** will return an error
    - **pivot_table** uses a function to aggregate the values (e.g. min, mean, max, etc.)

- **In essence pivot_table is a generalisation of pivot, which allows you to aggregate multiple values with the same destination in the pivoted table.**

In [13]:
#creating duplicate values
df['Item'] = ['Item0', 'Item0', 'Item0', 'Item1']
df

,Item,CType,USD,EU
0,Item0,Gold,1$,1€
1,Item0,Bronze,2$,2€
2,Item0,Gold,3$,3€
3,Item1,Silver,4$,4€


In [14]:
df.pivot(index = 'Item', columns = 'CType', values = 'USD')

ValueError: Index contains duplicate entries, cannot reshape

In [15]:
df.pivot_table(index = 'Item', columns = 'CType', values = 'USD', aggfunc = np.min)

CType,Bronze,Gold,Silver
Item,,,
Item0,2$,1$,NaN
Item1,NaN,NaN,4$


# Stack/Unstack

- pivoting is just a special case of **stacking** a dataframe

- stacking a sataFrame means moving (also rotating or pivoting) the innermost column index to become the innermost row index.

In [18]:
# Row Multi-Index
row_idx_arr = list(zip(['r0', 'r0'], ['r-00', 'r-01']))
row_idx = pd.MultiIndex.from_tuples(row_idx_arr)

# Column Multi-Index
col_idx_arr = list(zip(['c0', 'c0', 'c1'], ['c-00', 'c-01', 'c-10']))
col_idx = pd.MultiIndex.from_tuples(col_idx_arr)

df = pd.DataFrame(np.arange(6).reshape(2,3), index=row_idx, columns=col_idx).applymap(lambda x: (x // 3, x % 3))

In [19]:
df

c0              c1
           c-00    c-01    c-10
r0 r-00  (0, 0)  (0, 1)  (0, 2)
   r-01  (1, 0)  (1, 1)  (1, 2)

In [25]:
df.stack()

c0      c1
r0 r-00 c-00  (0, 0)     NaN
        c-01  (0, 1)     NaN
        c-10     NaN  (0, 2)
   r-01 c-00  (1, 0)     NaN
        c-01  (1, 1)     NaN
        c-10     NaN  (1, 2)

In [24]:
df.unstack()

c0                              c1        
      c-00            c-01            c-10        
      r-00    r-01    r-00    r-01    r-00    r-01
r0  (0, 0)  (1, 0)  (0, 1)  (1, 1)  (0, 2)  (1, 2)